# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

In [61]:
#| warning: false
#| message: false

library(dplyr)
library(tidyverse)

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

1. Randomly assign points to clusters.

2. Compute cluster centroids (means).

3. Reassign points to the nearest centroid.

4. Recompute centroids.

5. Repeat until convergence.

#### b) Create the builder function for step 1.

In [62]:
label_randomly <- function(n_points, n_clusters) {
  sample(((1:n_points) %% n_clusters)+1, n_points, replace = FALSE)
}

#### c) Create the builder function for step 2.

In [63]:
get_cluster_means <- function(data, labels){
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarise(across(everything(), mean), .groups = "drop") %>%
    mutate(label = label__) %>%
    arrange(label__)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [64]:
assign_cluster_fast <- function(data, means){
  data_matrix <- as.matrix(data)
  means_matrix <- as.matrix(means %>% dplyr::select(-label__))
  dii <- sort(rep(1:nrow(data), nrow(means)))
  mii <- rep(1:nrow(means), nrow(data))
  data_repped <- data_matrix[dii, ]
  means_repped <- means_matrix[mii, ]
  diff_squared <- (data_repped - means_repped)^2
  all_distances <- rowSums(diff_squared)
  tibble(dii=dii, mii=mii, distance=all_distances) %>%
    group_by(dii) %>%
    arrange(distance) %>%
    filter(row_number()==1) %>%
    ungroup() %>%
    arrange(dii) %>%
    pull(mii)
}

#### e) Create the builder function for step 4.

In [65]:
kmeans_done <- function(old_means, new_means, eps=1e-6) {
  om <- as.matrix(old_means)
  nm <- as.matrix(new_means)

  m <- mean(sqrt(rowSums((om-nm)^2)))

  if(m<eps) TRUE else FALSE
}

#### f) Combine them all into your own `kmeans` function.

In [66]:
mykmeans <- function(data, n_clusters, eps=1e-6) {
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE

  while(!done) {
    labels <- assign_cluster(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if (kmeans_done(old_means, new_means)){
      done <- TRUE
    }
    old_means <- new_means
  }
  list(labels = labels, means = new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set.

In [67]:
voltages <- read.csv("/content/voltages_df.csv")
head(voltages)

,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,X9.03614457831325,⋯,X240.963855421687,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1.031463,1.104665,0.8982475,0.4142208,-1.1490888,-1.078510,-1.002401,-0.9182083,-0.8215574,-0.7023741,⋯,-0.7392703,-0.7633694,-0.7792297,-0.7844340,-0.7779820,-0.7608812,-0.7369830,-0.7138199,-0.7014771,-0.7056029
2,-1.031463,1.246157,1.0948587,0.9039343,0.4654410,-1.160496,-1.112005,-1.0721319,-1.0385633,-1.0075872,⋯,-0.8859964,-0.8511675,-0.8064307,-0.7534558,-0.6954785,-0.6404759,-0.6105817,-0.6348313,-0.6767121,-0.7140939
3,-1.031463,1.216111,1.0557873,0.8417629,-0.5636836,-1.147653,-1.101783,-1.0645681,-1.0336197,-1.0051885,⋯,-0.9503509,-0.9122991,-0.8625269,-0.8016142,-0.7306757,-0.6527186,-0.5812047,-0.5875560,-0.6768023,-0.7206992
4,-1.031463,1.166244,0.9899628,0.7230858,-1.1806746,-1.125106,-1.077167,-1.0370309,-1.0027385,-0.9709488,⋯,-0.9498509,-0.9236047,-0.8896604,-0.8502120,-0.8086367,-0.7700917,-0.7418958,-0.7315532,-0.7409824,-0.7644406
5,-1.031463,1.230222,1.0746700,0.8733880,0.2116394,-1.153728,-1.106832,-1.0691075,-1.0383350,-1.0108187,⋯,-0.8710166,-0.8237315,-0.7590005,-0.6698582,-0.5061566,1.0975578,0.9348933,0.6673692,-1.1669718,-1.1047735
6,-1.031463,1.257650,1.1112886,0.9322788,0.6045620,-1.166325,-1.113488,-1.0696859,-1.0332517,-1.0009558,⋯,-0.9092342,-0.8715416,-0.8213803,-0.7589597,-0.6842115,-0.5958772,-0.4766488,1.1008087,0.9169321,0.5137345


#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`.

In [68]:
results <- mykmeans(voltages, n_clusters = 3)

results$labels
results$means

[1] 1 2 2 2 2 2 3 3 2 1 1 1 3 1 2 1 1 1 1 1 2 2 2 2 2 1 1 1 3 3 3 3 2 3 3 2 2
 [38] 1 3 2 1 1 1 3 2 3 3 2 3 3 2 3 1 1 3 1 2 1 3 2 1 1 2 1 3 3 2 3 1 3 1 2 3 3
 [75] 2 2 1 3 3 3 3 3 2 3 3 2 2 2 1 3 3 3 2 3 2 2 3 2 2 3 3 3 2 3 1 1 3 1 1 2 1
[112] 1 2 3 1 3 1 1 2 2 2 2 1 1 3 2 2 3 1 3 2 1 2 3 1 2 2 2 1 3 1 2 2 3 1 1 2 1
[149] 1 3 1 3 2 2 3 1 3 3 2 1 3 3 1 2 1 3 2 1 2 1 1 3 2 1 3 3 2 3 1 2 3 2 3 1 3
[186] 2 2 1 2 1 1 2 1 1 3 3 1 2 1 1 3 1 3 3 2 1 2 3 2 1 2 3 1 3 3 2 3 3 2 2 2 2
[223] 2 2 2 3 3 3 3 1 2 1 3 2 2 3 3 1 1 3 3 1 3 3 2 3 1 3 3 2 3 2 1 3 2 3 3 1 1
[260] 1 2 2 3 1 2 2 2 3 3 3 1 2 2 2 3 1 3 3 2 2 3 1 3 1 1 2 3 2 2 3 3 1 3 3 1 2
[297] 1 3 3 2 2 3 1 1 1 3 1 2 2 2 1 1 1 1 1 1 2 1 2 1 1 3 2 3 1 1 1 1 2 1 2 1 3
[334] 1 2 2 3 3 1 2 3 2 3 3 1 3 2 3 1 1 3 2 2 2 1 3 2 1 3 3 2 2 2 2 1 3 1 2 1 1
[371] 1 1 1 2 2 3 3 1 1 1 1 3 3 1 1 3 3 2 2 3 1 2 2 2 2 3 2 2 3 1 1 1 1 2 2 1 1
[408] 3 3 3 2 3 3 1 2 2 2 3 2 2 2 1 1 2 1 3 2 2 2 2 3 3 2 3 1 2 1 1 1 3 3 3 3 1
[445] 2 1 1 1 3 2 1 1 1 2 2 1 2 2 1 3 2 2 1 3 2 2 2 1 3 3 1 3 1 2 2 3 1 3 1 2 2
[482] 3 1 3 3 3 2 3 3 3 3 3 2 1 1 3 3 1 2 2 2 1 1 1 2 2 3 3 1 3 1 2 2 3 3 1 3 3
[519] 2 1 1 1 2 3 1 2 3 3 1 2 3 3 3 3 1 1 2 1 1 2 2 2 2 3 1 1 3 1 1 1 2 2 1 2 3
[556] 3 2 3 3 3 2 3 3 2 3 3 2 3 3 1 3 2 3 2 3 2 2 2 1 3 3 3 3 1 2 3 3 3 2 3 3 2
[593] 2 2 3 1 2 2 1 2 3 2 2 3 1 1 3 3 3 2 3 1 2 1 3 2 1 2 2 1 3 2 1 3 3 1 2 1 3
[630] 2 1 3 1 2 2 1 2 1 1 1 1 1 2 2 1 3 1 3 3 1 3 2 3 1 3 1 1 1 2 1 1 2 2 1 1 2
[667] 1 2 3 3 2 1 2 1 1 3 2 2 1 3 1 2 3 3 1 1 2 1 2 2 3 3 3 3 2 2 2 1 3 2 1 3 3
[704] 1 1 3 2 3 3 1 2 1 3 2 1 2 2 1 2 2 1 1 2 2 1 2 3 1 1 1 1 1 2 2 1 1 2 2 1 2
[741] 1 2 3 1 1 1 2 1 3 2 2 1 1 3 1 1 3 1 3 3 1 1 2 3 1 2 3 2 2 1 3 2 3 1 2 1 1
[778] 3 1 3 2 1 1 2 3 2 1 2 1 3 1 1 2 2 2 1 2 1 3 2 2 2 2 1 1 3 2 1 1 1 1 3 3 1
[815] 3 1 2 1 2 3 2 2 2 2 2 3 3 2 3 3 3 3 3 3 2 1 1 2 3 3 1 2 3 3 3 3 1 1 2 3 2
[852] 1 3 3 3 3 1 1 3 3 3 3 3 1 2 3 2 1 2 2 3 1 3 3 1 1 2 1 1 1 3 2 3 2 2 1 1 3
[889] 3 2 1 3 3 2 2 2 2 1 3 3

label__,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,⋯,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250,label
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1.031463,0.9381238,0.7619864,0.3631543,-1.1179412,-1.051145,-0.9766807,-0.8694758,-0.6892375,⋯,-0.8070676,-0.8182598,-0.8207339,-0.8132928,-0.7969549,-0.77567272,-0.75689256,-0.7496483,-0.7570393,1
2,-1.031463,1.2439759,1.0924697,0.9004440,0.3011754,-1.159714,-1.1098127,-1.0685484,-1.0338649,⋯,-0.8732292,-0.8234477,-0.7607812,-0.6682618,-0.3380864,-0.04693168,0.02820486,-0.4113500,-0.8115784,2
3,-1.031463,1.3093239,1.1616772,0.9787498,0.6481497,-1.168610,-1.1196122,-1.0590962,-0.9943176,⋯,0.8337474,0.7125412,-0.2659209,-1.0409179,-1.0587745,-1.01359887,-0.96467777,-0.9151047,-0.8610245,3


#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`.
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [69]:
voltages_matrix <- as.matrix(voltages)

results <- kmeans(voltages_matrix, centers = 3)

results$cluster
results$centers

[1] 2 3 3 3 3 1 2 2 3 2 2 2 2 2 3 2 2 2 2 2 1 3 3 3 3 2 2 2 2 2 2 2 3 2 2 3 3
 [38] 2 2 1 2 2 2 2 1 2 2 3 2 2 1 2 2 2 2 2 1 2 2 3 2 2 3 2 2 2 1 2 2 2 2 3 2 2
 [75] 3 1 2 2 2 2 2 2 3 2 2 3 3 3 2 2 2 2 3 2 3 1 2 3 3 2 2 2 3 2 2 2 2 2 2 1 2
[112] 2 1 2 2 2 2 2 3 3 1 3 2 2 2 1 3 2 2 2 3 2 3 2 2 1 3 1 2 2 2 3 3 2 2 2 3 2
[149] 2 2 2 2 3 3 2 2 2 2 1 2 2 2 2 3 2 2 3 2 3 2 2 2 3 2 2 2 3 2 2 1 2 3 2 2 2
[186] 1 1 2 1 2 2 1 2 2 2 2 2 1 2 2 2 2 2 2 3 2 3 2 1 2 1 2 2 2 2 3 2 2 3 3 3 1
[223] 3 3 1 2 2 2 2 2 3 2 2 3 3 2 2 2 2 2 2 2 2 2 3 2 2 2 2 1 2 3 2 2 1 2 2 2 2
[260] 2 3 3 2 2 3 3 3 2 2 2 2 1 1 1 2 2 2 2 1 3 2 2 2 2 2 1 2 3 3 2 2 2 2 2 2 1
[297] 2 2 2 3 3 2 2 2 2 2 2 1 3 1 2 2 2 2 2 2 3 2 3 2 2 2 3 2 2 2 2 2 3 2 1 2 2
[334] 2 1 3 2 2 2 3 2 3 2 2 2 2 3 2 2 2 2 3 1 3 2 2 3 2 2 2 1 3 3 3 2 2 2 1 2 2
[371] 2 2 2 3 1 2 2 2 2 2 2 2 2 2 2 2 2 1 3 2 2 3 3 1 3 2 1 3 2 2 2 2 2 3 3 2 2
[408] 2 2 2 3 2 2 2 3 1 1 2 3 3 3 2 2 1 2 2 3 1 3 1 2 2 1 2 2 1 2 2 2 2 2 2 2 2
[445] 3 2 2 2 2 3 2 2 2 1 3 2 3 3 2 2 3 3 2 2 3 3 3 2 2 2 2 2 2 3 3 2 2 2 2 3 3
[482] 2 2 2 2 2 3 2 2 2 2 2 3 2 2 2 2 2 3 3 3 2 2 2 3 3 2 2 2 2 2 3 1 2 2 2 2 2
[519] 3 2 2 2 1 2 2 1 2 2 2 3 2 2 2 2 2 2 3 2 2 3 3 3 1 2 2 2 2 2 2 2 1 3 2 3 2
[556] 2 3 2 2 2 3 2 2 3 2 2 3 2 2 2 2 1 2 3 2 1 1 3 2 2 2 2 2 2 3 2 2 2 3 2 2 3
[593] 3 1 2 2 3 3 2 3 2 3 1 2 2 2 2 2 2 1 2 2 1 2 2 3 2 3 3 2 2 3 2 2 2 2 3 2 2
[630] 1 2 2 2 1 3 2 3 2 2 2 2 2 1 1 2 2 2 2 2 2 2 1 2 2 2 2 2 2 3 2 2 3 3 2 2 3
[667] 2 1 2 2 3 2 3 2 2 2 1 3 2 2 2 3 2 2 2 2 3 2 1 3 2 2 2 2 3 1 1 2 2 3 2 2 2
[704] 2 2 2 3 2 2 2 1 2 2 3 2 1 3 2 3 3 2 2 3 1 2 1 2 2 2 2 2 2 3 3 2 2 3 3 2 1
[741] 2 1 2 2 2 2 3 2 2 3 1 2 2 2 2 2 2 2 2 2 2 2 3 2 2 1 2 1 3 2 2 3 2 2 1 2 2
[778] 2 2 2 3 2 2 3 2 1 2 1 2 2 2 2 3 3 3 2 3 2 2 3 3 3 3 2 2 2 1 2 2 2 2 2 2 2
[815] 2 2 1 2 3 2 1 3 3 3 3 2 2 3 2 2 2 2 2 2 3 2 2 3 2 2 2 3 2 2 2 2 2 2 3 2 3
[852] 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 3 2 1 3 2 2 2 2 2 2 3 2 2 2 2 1 2 3 3 2 2 2
[889] 2 3 2 2 2 3 3 3 3 2 2 2

,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,X9.03614457831325,⋯,X240.963855421687,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250
1,-1.031463,1.245873,1.0950402,0.9049394,0.3511373,-1.160528,-1.110436,-1.069043,-1.0343093,-1.0027217,⋯,-0.9093694,-0.87200867,-0.82253175,-0.7606134,-0.6785901,-0.3916473,-0.08367886,0.02967317,-0.3416634,-0.7627508
2,-1.031463,1.123724,0.9618318,0.6709520,-0.2348958,-1.109877,-1.048146,-0.964286,-0.8417775,-0.7449379,⋯,-0.2268060,0.01333991,-0.05285934,-0.5433274,-0.9271054,-0.9278647,-0.89463579,-0.86078516,-0.8323765,-0.8090319
3,-1.031463,1.243124,1.0913149,0.8984242,0.2787287,-1.159348,-1.109533,-1.068326,-1.0336653,-1.0020231,⋯,-0.9113662,-0.87377754,-0.82385929,-0.7608566,-0.6636216,-0.3140227,-0.03042208,0.02754518,-0.4426584,-0.8335155


#### d) Are your labels/clusters the same? If not, why? Are your means the same?

No, the labels and clusters are not the same. Because both the built in function and the function I created started with random initial centroids, the cluster numbering and assignment can change each time the code is run, so the cluster might not match. Cluster means are quite similar but they are not identical as the different random starting points can lead to slightly different final centroids and they both might not converge to the same minimum. However since the numeric values are close, the clusters would capture similar patterns, leading to similar mean values.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

The process of using a for loop to assign clusters for kmeans by looping through each data point in the dataset, calculating the distance to the current cluster centroids, finding the closest centroid for that point, and assigning the point to the cluster of the closest centroid.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

Vectorized assignment calculates the distance for all points to all centroids in one step using matrix operations. This then assigns each point to the nearest centroid without explicit loops.

#### c) State which (for loops or vectorizing) is more efficient and why.

Vectorization is more efficient because it uses optimized code. Using for loops runs slower as it needs to execute each iteration sequentially.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

kmeans fails when the clusters are not roughly spherical or equally sized, clusters have very different densities or variances, data is not in the vector space, or there are extreme outliers. kmeans uses the assumption that we are dealing with vectorial data, clusters are spherical, and each point only belongs to one cluster.

## Question 5
#### What assumption do Guassian mixture models make?

The Gaussian mixture models assume that the data are generated from a mixture of several Gaussian distributions, each representing a cluster. Each cluster is modeled after a Gaussian shaped region in the data space. These mixture models allow each Gaussian component to have its own mean and covariance, meaning that clusters can vary in shape, size, and orientation. They also assume that each data point belong to clusters with certain probabilities rather than being assigned to a single cluster absolutely, kind of like the soft clustering approach that we have with fuzzy k-means.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

Spectral clustering assumes that two points are more likely to be in the same cluster if they are close to one another. It relies on a similarity measure between points (aka distance), so it can work with nonlinear, irregularly shaped cluster, and non-vector data. Allows us to handle more complex data structures such as graphs or networks, where relationships are defined by connectivity or similarity.

## Question 7
#### Define the gap statistic method. What do we use it for?

Gap statistic is a method used to determine the optimal number of k clusters in a dataset. Compares within-cluster dispersion for different values of k to the null distribution, which is the dataset with no clustering. It measures how much better the observed clustering structure is compared to the clustering that is expected by chance. We can use this to find the value of k that maximizes the gap statistic, which indicates the point where adding more clusters no longer provides a significant improvement in clustering quality.